Next Word Predictor


In [5]:
!pip install opendatasets

In [6]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/ronikdedhia/next-word-prediction")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jhasm06
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/ronikdedhia/next-word-prediction


100%|██████████| 228k/228k [00:00<00:00, 441MB/s]

In [7]:
file_path = './next-word-prediction/1661-0.txt'

# Read the content of the file into the 'document' variable
with open(file_path, 'r', encoding='utf-8') as f:
    document = f.read()

print(f"Dataset loaded into 'document' variable. First 500 characters:\n{document[:500]}...")

Dataset loaded into 'document' variable. First 500 characters:
﻿
Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release Date: November 29, 2002 [EBook #1661]
Last Updated: May 20, 2019

Language: English

Charac...


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader, Dataset
import nltk


In [9]:
#tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
tokens=word_tokenize(document.lower())

In [11]:
#Vocab

vocab={'<unk>':0}
for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token]=len(vocab)

vocab

{'<unk>': 0,
 '\ufeff': 1,
 'project': 2,
 'gutenberg': 3,
 "'s": 4,
 'the': 5,
 'adventures': 6,
 'of': 7,
 'sherlock': 8,
 'holmes': 9,
 ',': 10,
 'by': 11,
 'arthur': 12,
 'conan': 13,
 'doyle': 14,
 'this': 15,
 'ebook': 16,
 'is': 17,
 'for': 18,
 'use': 19,
 'anyone': 20,
 'anywhere': 21,
 'at': 22,
 'no': 23,
 'cost': 24,
 'and': 25,
 'with': 26,
 'almost': 27,
 'restrictions': 28,
 'whatsoever': 29,
 '.': 30,
 'you': 31,
 'may': 32,
 'copy': 33,
 'it': 34,
 'give': 35,
 'away': 36,
 'or': 37,
 're-use': 38,
 'under': 39,
 'terms': 40,
 'license': 41,
 'included': 42,
 'online': 43,
 'www.gutenberg.net': 44,
 'title': 45,
 ':': 46,
 'author': 47,
 'release': 48,
 'date': 49,
 'november': 50,
 '29': 51,
 '2002': 52,
 '[': 53,
 '#': 54,
 '1661': 55,
 ']': 56,
 'last': 57,
 'updated': 58,
 '20': 59,
 '2019': 60,
 'language': 61,
 'english': 62,
 'character': 63,
 'set': 64,
 'encoding': 65,
 'utf-8': 66,
 '*': 67,
 'start': 68,
 'produced': 69,
 'an': 70,
 'anonymous': 71,
 'volunt

In [12]:
len(vocab)

9399

In [13]:
#extract sentences from data
input_sentences=document.split('\n')

In [14]:
input_sentences

['\ufeff',
 "Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle",
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.net',
 '',
 '',
 'Title: The Adventures of Sherlock Holmes',
 '',
 'Author: Arthur Conan Doyle',
 '',
 'Release Date: November 29, 2002 [EBook #1661]',
 'Last Updated: May 20, 2019',
 '',
 'Language: English',
 '',
 'Character set encoding: UTF-8',
 '',
 '*** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***',
 '',
 '',
 '',
 'Produced by an anonymous Project Gutenberg volunteer and Jose Menendez',
 '',
 '',
 '',
 'cover',
 '',
 '',
 '',
 'The Adventures of Sherlock Holmes',
 '',
 '',
 '',
 'by Arthur Conan Doyle',
 '',
 '',
 '',
 'Contents',
 '',
 '',
 '   I.     A Scandal in Bohemia',
 '   II.    The Red-Head

In [15]:
def text_to_indices(sentence,vocab):
  numerical_sentence=[]

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])
  return numerical_sentence

In [16]:
input_numerical_sentences=[]

for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()),vocab))

In [17]:
training_sequence=[]
for sentence in input_numerical_sentences:
  for i in range(1,len(sentence)):
    training_sequence.append(sentence[:i+1])



In [18]:
len_list=[]
for sequence in training_sequence:
  len_list.append(len(sequence))

max(len_list)

29

In [19]:
padded_training_sequence=[]
for sequence in training_sequence:
  padded_training_sequence.append([0]*(max(len_list)-len(sequence))+sequence)

In [20]:
padded_training_sequence=torch.tensor(padded_training_sequence,dtype=torch.long)

In [21]:
padded_training_sequence


tensor([[   0,    0,    0,  ...,    0,    2,    3],
        [   0,    0,    0,  ...,    2,    3,    4],
        [   0,    0,    0,  ...,    3,    4,    5],
        ...,
        [   0,    0,    0,  ...,  588, 1172,  417],
        [   0,    0,    0,  ..., 1172,  417, 9112],
        [   0,    0,    0,  ...,  417, 9112,   30]])

In [22]:
X=padded_training_sequence[:,:-1]
y=padded_training_sequence[:,-1]

In [23]:
class CustomDataset(Dataset):
  def __init__(self,X,y):
    self.X=X
    self.y=y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__ (self,idx):
    return self.X[idx],self.y[idx]

In [24]:
dataset=CustomDataset(X,y)

In [25]:
len(dataset)

120233

In [26]:
dataloader=DataLoader(dataset,batch_size=32,shuffle=True)

In [27]:
for input,output in dataloader:
  print(input,output)

Streaming output truncated to the last 5000 lines.
            0,    0,    0,  450,   25,   31,   32, 3096,  242, 2910,   80,    5,
         1251,   30,  126,  127],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,  450,   25,  359,  126, 1645,  239, 1646,  657,  213,  202,
            5,  395,  687,  126],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  215],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0, 2247,   10],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,  450,  767],
        [   0,    0,    0,    0,    0,    0,    0,

In [28]:
total_lines = len(document.split('\n'))
print(f"The dataset contains {total_lines} lines.")

The dataset contains 12311 lines.


# Architecture

In [29]:
class LSTMModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,100)
    self.lstm=nn.LSTM(100,150,batch_first=True)
    self.fc=nn.Linear(150,vocab_size)

  def forward(self,x):
    embedded=self.embedding(x)
    intermediate_hidden_states,(final_hidden_state,final_cell_state)=self.lstm(embedded)
    output=self.fc(final_hidden_state.squeeze(0))
    return output

In [30]:
model=LSTMModel(len(vocab))

In [31]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
model.to(device)

LSTMModel(
  (embedding): Embedding(9399, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=9399, bias=True)
)

In [34]:
epochs=20
learning_rate=0.001

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [35]:
#Training Loop

for epoch in range(epochs):
  total_loss=0
  for batch_x,batch_y in dataloader:
    batch_x,batch_y=batch_x.to(device),batch_y.to(device)
    optimizer.zero_grad()
    output=model(batch_x)
    loss=criterion(output,batch_y)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()
  print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 20573.2777
Epoch 2, Loss: 17540.2422
Epoch 3, Loss: 15975.3791
Epoch 4, Loss: 14649.2678
Epoch 5, Loss: 13466.4095
Epoch 6, Loss: 12424.3698
Epoch 7, Loss: 11482.8595
Epoch 8, Loss: 10649.2513
Epoch 9, Loss: 9888.0872
Epoch 10, Loss: 9211.6746
Epoch 11, Loss: 8597.8408
Epoch 12, Loss: 8042.0043
Epoch 13, Loss: 7552.8493
Epoch 14, Loss: 7110.8117
Epoch 15, Loss: 6706.8071
Epoch 16, Loss: 6344.5180
Epoch 17, Loss: 6026.5839
Epoch 18, Loss: 5726.9116
Epoch 19, Loss: 5464.3318
Epoch 20, Loss: 5225.2958


In [36]:
#prediction

def prediction(model,vocab,text):
  #tokenize
  tokenized_text=word_tokenize(text.lower())


  #Text->numerical
  numerical_text=text_to_indices(tokenized_text,vocab)

  #Padding
  padded_text=torch.tensor([0]*(61-len(numerical_text))+numerical_text,dtype=torch.long).unsqueeze(0)

  #send to model
  output=model(padded_text)

  #predicted index
  value,index=torch.max(output,dim=1)

  #merge with text
  return text+""+list(vocab.keys())[index]

In [38]:
import time
num_tokens=10
input_text="I know"

for i in range(num_tokens):
  output_text=prediction(model,vocab,input_text)
  print(output_text)
  input_text=output_text
  time.sleep(0.5)

I knowhim
I knowhim.
I knowhim.”
I knowhim.”he
I knowhim.”hethrew
I knowhim.”hethrew.
I knowhim.”hethrew.”
I knowhim.”hethrew.”he
I knowhim.”hethrew.”hehimself
I knowhim.”hethrew.”hehimself.


In [39]:
model.eval()  # Set the model to evaluation mode
correct_predictions = 0
total_predictions = 0

with torch.no_grad():  # Disable gradient calculations during evaluation
    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        output = model(batch_x)
        _, predicted = torch.max(output.data, 1)
        total_predictions += batch_y.size(0)
        correct_predictions += (predicted == batch_y).sum().item()

accuracy = 100 * correct_predictions / total_predictions
print(f'Accuracy of the model on the dataset: {accuracy:.2f}%')

model.train()  # Set the model back to training mode if further training is planned

Accuracy of the model on the dataset: 73.38%


LSTMModel(
  (embedding): Embedding(9399, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=9399, bias=True)
)